In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss, brier_score_loss, precision_score, recall_score, f1_score
from datetime import date
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'D:/CS 5100/MoA/'
output_folder = 'D:/CS 5100/Output/'

# fix the random seed 
xseed = 43

# number of folds for cv
nfolds = 5

# number of components to retain from PCA decomposition
nof_comp = 300

model_name = 'svm'

In [3]:
xtrain = pd.read_csv(data_folder + 'train_features.csv')
xtest = pd.read_csv(data_folder + 'test_features.csv')
ytrain = pd.read_csv(data_folder + 'train_targets_scored.csv')

In [4]:
# due to small cardinality of all values, it's faster to handle categoricals that way,

print(set(xtrain['cp_time']), set(xtest['cp_time']) )

# cp_time
xtrain['cp_time_24'] = (xtrain['cp_time'] == 24) + 0
xtrain['cp_time_48'] = (xtrain['cp_time'] == 48) + 0
xtest['cp_time_24'] = (xtest['cp_time'] == 24) + 0
xtest['cp_time_48'] = (xtest['cp_time'] == 48) + 0
xtrain.drop('cp_time', axis = 1, inplace = True)
xtest.drop('cp_time', axis = 1, inplace = True)

# cp_dose
print(set(xtrain['cp_dose']), set(xtest['cp_dose']) )
xtrain['cp_dose_D1'] = (xtrain['cp_dose'] == 'D1') + 0
xtest['cp_dose_D1'] = (xtest['cp_dose'] == 'D1') + 0
xtrain.drop('cp_dose', axis = 1, inplace = True)
xtest.drop('cp_dose', axis = 1, inplace = True)

# cp_type
xtrain['cp_type_control'] = (xtrain['cp_type'] == 'ctl_vehicle') + 0
xtest['cp_type_control'] = (xtest['cp_type'] == 'ctl_vehicle') + 0
xtrain.drop('cp_type', axis = 1, inplace = True)
xtest.drop('cp_type', axis = 1, inplace = True)

{24, 48, 72} {24, 48, 72}
{'D1', 'D2'} {'D1', 'D2'}


In [5]:
# prepare split
kf = KFold(n_splits = nfolds)

# separation
id_train = xtrain['sig_id']; id_test = xtest['sig_id']
ytrain.drop('sig_id', axis = 1, inplace = True) 
xtrain.drop('sig_id', axis = 1, inplace = True)
xtest.drop('sig_id', axis = 1, inplace = True)

# storage matrices for OOF / test predictions
prval = np.zeros(ytrain.shape)
prfull = np.zeros((xtest.shape[0], ytrain.shape[1]))

In [6]:
# base model definition through sklearn Pipeline
pca = PCA(n_components = nof_comp)
svm0 = SVR(C = 0.1)

base_model = Pipeline(steps=[('pca', pca), ('svm', svm0)])

mo_base = MultiOutputRegressor(base_model, n_jobs=-1)

In [7]:
for (ff, (id0, id1)) in enumerate(kf.split(xtrain)):
     
    x0, x1 = xtrain.loc[id0], xtrain.loc[id1]
    y0, y1 = np.array(ytrain.loc[id0]), np.array(ytrain.loc[id1])
    
    # stupid fix for empty columns - LogisticRegression blows up otherwise 
    # (the problem occurs for two folds only, each time for a single column)
    # yes, i know it's ugly
    check_for_empty_cols = np.where(y0.sum(axis = 0) == 0)[0]
    if len(check_for_empty_cols):
        y0[0,check_for_empty_cols] = 1
    
    # fit model
    mo_base.fit(x0,y0)
    
    prv = mo_base.predict(x1)
    prf = mo_base.predict(xtest)
    # generate the prediction
    prval[id1,:] = prv
    prfull += prf/nfolds
    
    
    print('fold '+str(ff) + ': completed')

fold 0: completed
fold 1: completed
fold 2: completed
fold 3: completed
fold 4: completed


In [8]:
column_list = ytrain.columns

prval_cal = np.zeros(ytrain.shape)
prfull_cal = np.zeros((xtest.shape[0], ytrain.shape[1]))



for (ff, (id0, id1)) in enumerate(kf.split(xtrain)):
     
    for ii in range(0, ytrain.shape[1]):
        
        xname = column_list[ii]
        
        x0, x1 = prval[id0,ii], prval[id1,ii]
        y0, y1 = np.array(ytrain)[id0,ii], np.array(ytrain)[id1,ii]
       
        if sum(y0) == 0:
            y0[0] = 1
            
        basemodel = LogisticRegression()        
        basemodel.fit(x0.reshape(-1,1), y0)
        prv = basemodel.predict_proba(x1.reshape(-1,1))[:,1]
        prf = basemodel.predict_proba(np.array(prfull)[:,ii].reshape(-1,1))[:,1]
        
        prval_cal[id1, ii] = prv
        prfull_cal[:, ii] += prf/nfolds

    print(ff)

0
1
2
3
4


In [9]:
# compare performance pre- and post- calibration
metrics1 = []
metrics2 = []


for ii in range(0,ytrain.shape[1]):
    loss1 = log_loss(np.array(ytrain)[:, ii], prval[:, ii])
    metrics1.append(loss1)
    loss2 = log_loss(np.array(ytrain)[:, ii], prval_cal[:, ii])
    metrics2.append(loss2)
    
print('raw: ' + str(np.mean(metrics1)) )
print('cal: ' + str(np.mean(metrics2)))

raw: 0.10190956138382551
cal: 0.017780453500434706


In [10]:
prval_cal = pd.DataFrame(prval_cal)
prfull_cal = pd.DataFrame(prfull_cal)
prval_cal.columns = ytrain.columns
prfull_cal.columns = ytrain.columns

prval_cal['sig_id'] = id_train
prfull_cal['sig_id'] = id_test

In [11]:
metrics = []
for _target in ytrain.columns:
    metrics.append(log_loss(ytrain.loc[:, _target], prval_cal.loc[:, _target]))
print(f'OOF Metric: {np.round(np.mean(metrics),4)}')

OOF Metric: 0.0178


In [12]:
xcols = list(ytrain.columns); xcols.insert(0, 'sig_id')
prval_cal = prval_cal[xcols]; prfull_cal = prfull_cal[xcols]


todate = date.today().strftime("%d%m")
print(todate)

# files for combination
# prval_cal.to_csv(output_folder + 'prval_'+model_name+'_'+todate+'.csv', index = False)
# prfull_cal.to_csv(output_folder + 'prfull_'+model_name+'_'+todate+'.csv', index = False)
# actual submission
prfull_cal.to_csv(output_folder + 'submission.csv', index = False)

1212


In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)
    
    print(train_sizes,train_scores,test_scores)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

#X, y = load_digits(return_X_y=True)

#title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
#cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

#estimator = GaussianNB()
#plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.7, 1.01),cv=cv, n_jobs=4)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = mo_base
plot_learning_curve(estimator, title, xtrain, ytrain, axes=axes[:, 1], ylim=(0, 1.01), cv=cv, n_jobs=4)

plt.show()